# Henry Markowitz et les quarante portefeuilles 

In [1]:
%%javascript
MathJax.Hub.Config({
    TeX: { equationNumbers: { autoNumber: "AMS" } }
});

<IPython.core.display.Javascript object>

In [36]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import optimize
from casadi import *

L’optimisation de portefeuille a pour objectif de définir un choix "optimal" d’investissement d’un capital
dans un ensemble de n actifs. Pour ce faire, on note
$x_i$
l’investissement dans le i-ème actif, de sorte que le
vecteur $x \in \mathbb{R}^n$
 décrit l’allocation globale du portefeuille à travers l’ensemble des actifs. Ceci génère un
rendement $r$ pour l’ensemble du portefeuille, incertain,
dont on cherche à minimiser le risque.

On considère le vecteur $p \in \mathbb{R}^n$ de variation du prix
des actifs sur le long terme, supposé gaussien, de
moyenne $\bar{p}$ et de matrice de covariance $\Sigma$, semi-définie
positive. Pour tenir compte du budget à investir, on a
$$ 1^\top x = 1 \tag{1} $$
où $1^\top$ désigne le vecteur ligne $(1, . . . , 1) \in \mathbb{R}^n$.  Par
ailleurs, pour un rendement 2} $r$ donné, on sait que
$$ \bar{p}^\top x = r \tag{2} $$
et que le risque associé à un investissement $x$ peut
s’exprimer comme sa variance
$$ x^\top \Sigma x \tag{3}$$

## Etude de l’expression d’origine par Markowitz

**Question 1** Dans ce problème, on se donne un portefeuille. On veut investir plus ou moins dans un ensemble de $n$ actifs. Pour cela, une limite : on ne peut pas dépasser son budget initial. On peut alors normaliser ces investissements, en se fixant un budget normalisé de $1$. Dans ce cas, notons $(x_1,...,x_n)$ le vecteur des différentes allocations. Si pour $i$ donné on a $x_i$ positif, on espère que l'actif $i$ augmente pour le revendre. Au contraire, si $x_i$ est négatif, on appelle cela une vente à découvert : on espère que l'actif $i$ baisse en valeur pour le racheter, et ainsi avoir une plus-value. Avec notre budget normalisé de $1$, on a logiquement :
$$ \sum_{i = 1}^n x_i = 1$$
ce qui revient, en vectoriel, à l'équation $(1)$. L'équation $(2)$ induit que l'on peut lier un placement $x$ et son rendement attendu $r$, ou inversement un rendement fixé et un placement valable, grâce au vecteur des variations moyennes de prix $\bar{p}$ : en effet, en pondérant nos placements par ces facteurs stochastiques, on prévoit la variation de notre capital (normalisée). Enfin, en se donnant un rendement à atteindre $r$, on veut essayer de minimiser le risque que notre rendement réel s'écarte de ce que l'on veut idéalement avec l'équation $(2)$. Ce risque, c'est la variance liée à $p$, c'est à dire $x^\top \Sigma x$.

**Question 2** On cherche désormais à écrire ce problème comme un problème d'optimisation, sous la forme
$$\min_{c_{eq}(z) = 0} f(z) \tag{4}$$
Dans ce cas, c'est le risque lié à la variance qu'il faut minimiser ici, sous les contraintes des équations $(1)$ et $(2)$. Pour coller au problème, on a $n$ variables, autant que d'actifs dans lesquels il est possible d'investir. Notre $z$ est évidemment le vecteur $x$ des allocations. Ce qu'on doit minimiser, c'est la fonction $f : x \in \mathbb{R}^n \mapsto x^\top \Sigma x \in \mathbb{R}$.
Notre contrainte $c_{eq}$ est telle que :
$$ c_{eq} : x \in \mathbb{R}^n \mapsto (1^\top x -1,  \bar{p}^\top x - r) \in \mathbb{R}^2 $$

**Question 3** L'utilisation du max vectoriel permet de ne sélectionner que les investissements négatifs, ou ventes à découvert, ou positions courtes. Ainsi, on somme les ventes à découvert, et on oblige la valeur absolue de cette somme à ne pas dépasser une certaine constante $s_M$ donnée. En faisant cela, le total des allocations d'investissements positifs est au moins de $1-s_M$. La difficulté, c'est que cette contrainte que l'on rajoute,
$$ 1^\top \max(-x,0) \leq s_M \tag{5} $$

**Question 4** On introduit $s \in \mathbb{R}^n$, tel que :
$$ s \geq -x, \ \ s\geq 0, \ \ 1^\top s \leq s_M \tag{6}$$
Alors, puisque $s \geq -x$, et $s \geq 0$, à fortiori $s \geq \max(-x,0)$ (avec la définition terme à terme). Mais comme la somme des composantes de $s$ est plus petite que $s_M$, ici aussi à fortiori la somme des composantes de $\max(-x,0)$ est plus petite que $s_M$, ce qui donne bien la condition $(5)$.

*A faire : modifier implémentation du problème*

## Etude numérique du problème de Markowitz

**Question 5** On a ici affaire à un problème d'optimisation quadratique différentiable, sous contraintes de type égalité. Il faut utiliser de la programmation quadratique.

**Question 6** On commence par définir les paramètres d'intérêt, puis la fonction à minimiser, et les contraintes à étudier.

In [37]:
p1 = 0.05
p2 = 0.15
p3 = 0.30
sig1 = 0.10
sig2 = 0.30
sig3 = 0.80
rho = 0.1
r = 0.1
pmoy = np.array([p1,p2,p3])
Sig = np.array([[sig1**2,rho*sig1*sig2,0.],[rho*sig1*sig2,sig2**2,0.],[0.,0.,sig3**2]])
Sig

array([[0.01 , 0.003, 0.   ],
       [0.003, 0.09 , 0.   ],
       [0.   , 0.   , 0.64 ]])

In [44]:
def risque(x):
    ecart = np.matmul(x.T,np.matmul(Sig,x))
    return ecart

In [45]:
uno = np.array([1,1,1])
risque(uno)

0.7460000000000001

In [53]:
def c1(x):
    c = x[0] + x[1] + x[2] - 1
    return c

def c2(x):
    c = pmoy[0]*x[0] + pmoy[1]*x[1] + pmoy[2]*x[2] - r
    return c

In [54]:
c1(uno), c2(uno)

(2, 0.4)

In [59]:
opti = casadi.Opti();
n = 3
x1 = opti.variable(n)
opti.minimize(risque(x1))
opti.subject_to(c1(x1)==0)
opti.subject_to(c2(x1)==0)
x10 = np.array([1/3, 1/3, 1/3])
opti.set_initial(x1,x10)
opti.solver('ipopt');
sol1 = opti.solve();
solx1 = sol1.value(x1)
print(solx1)

This is Ipopt version 3.12.3, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:        6
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:        6

Total number of variables............................:        3
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:        2
Total number of inequality constraints...............:        0
        inequality constraints with only lower bounds:        0
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0 8

In [64]:
solx1[0] + solx1[1] + solx1[2], pmoy[0]*solx1[0] + pmoy[1]*solx1[1] + pmoy[2]*solx1[2], risque(solx1)

(1.0, 0.1, 0.017067775467775475)

**Question 10** On cherche ici à minimiser le risque moins le rendement, donc à maximiser le rendement moins le risque ! Dans le cas $\mu = 0$, il s'agit de minimiser simplement $-\bar{p}^\top x$ ; soit de maximiser le rendement $r$ !